In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp
#initiate variables 
has_heartbeat = False
cohort0 = False
cohort2 = False
onix_analog_clock_downsampled = False
onix_analog_framecount_upsampled = False
common_resampled_rate = 1000 #in Hz
unit_conversions = False #
save_full_asynchronous_data = True #saves alldata before resampling
optical_filter_Hz=40 #filter cutoff for Optical tracking and encoder signals

#Cohort 1 vestibular mismatch, multiple OnixDigital files 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2718-2024-12-12T13-28-14') #multiple onix_digital file

#Cohort 1 vestibular mismatch, with clock accumulation issue marked on google sheet, seems fine though
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2719-2024-12-12T13-59-38') #multiple onix_digital file

#Cohort 1 vestibular mismatch
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2717-2024-12-12T13-00-21')

#Cohort 1 visual mismatch 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2718-2024-12-10T12-57-02') 

#Cohort 1 visual mismatch THIS
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')

#Cohort 0 (no OnixHarp in this Cohort)
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort0/Cohort0_GCaMP_example/B3M3xx-2024-08-08T10-05-26')
#cohort0 = True

#Cohort 2 (Cohort 1 animal) 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_test/2025-02-13T12-41-57')
#has_heartbeat = True

photometry_path = data_path.parent / f"{data_path.name}_processedData" / "photometry"
save_path = data_path.parent / f"{data_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])

#h1_datafolder = data_path / 'HarpDataH1' #only if reading separate registers
#h2_datafolder = data_path / 'HarpDataH2' #only if reading separate registers
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
#h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
#h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)

#create loaders 
session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
#photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.VideoReader("VideoData1")
video_reader2 = utils.VideoReader("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)

if cohort0:
    print ("Loading OnixAnalogFrameClock")
    onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
    
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData and converting to boolean photodiode array")
photodiode = utils.read_OnixAnalogData(data_path, channels = [0], binarise=False, method='adaptive', refractory = 0.5, flip=True, verbose=True) #method adaptive or threshold (which is hard threshold at 120), refractory in seconds (aasuming 100kHz sampling rate) to avoid multiple detections

print ("✅ Done Loading")

In [ ]:
experiment_events

In [ ]:
def print_event_times(event_string):
    event_times = experiment_events[experiment_events['Event'].str.contains(event_string)].index
    relative_times = (event_times - experiment_events.index[0]).total_seconds()
    print(relative_times)

# Example usage
myevents = "No halt"
myevents = "Apply halt: 2s"
print_event_times(myevents)

In [ ]:
# Define the start and end time in seconds
start_time = 400         # start time in seconds
end_time = 500    # end time in seconds
sampling_rate = 100000
# Calculate the number of samples to plot
start_sample = int(start_time * sampling_rate)
end_sample = int(end_time * sampling_rate)

# Extract the relevant portion of the photodiode array
photodiode_segment = photodiode[start_sample:end_sample]

# Create a time array for the x-axis
time_array = np.linspace(start_time, end_time, end_sample - start_sample)

# Plot the photodiode array using Matplotlib
plt.figure(figsize=(10, 5))
plt.plot(time_array, photodiode_segment.flatten(), label='Photodiode Signal')
plt.title('Photodiode Array')
plt.xlabel('Time (seconds)')
plt.ylabel('Photodiode Signal')
plt.show()
